In [ ]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from astropy.visualization import ZScaleInterval, ImageNormalize, LinearStretch
import glob, os

# === Path setup ===
BASE_DIR = "data"

# Calibration folders
CALIB_DIR = os.path.join(BASE_DIR, "calibration")
BIAS_DIR = os.path.join(CALIB_DIR, "bias")
FLAT_DIR = os.path.join(CALIB_DIR, "flats")
MASTER_DIR = os.path.join(CALIB_DIR, "master")

# Science frames
TRANSIT_DIR = os.path.join(BASE_DIR, "raw/transit")
STANDARD_DIR = os.path.join(BASE_DIR, "raw/standard_stars")

# Output 
OUTPUT_TRANSIT = os.path.join(BASE_DIR, "reduced/transit")
OUTPUT_STANDARDS = os.path.join(BASE_DIR, "reduced/standard_stars")

def show_fits(data, title=""):
    """
    stolen from the obs astro code but like the backend code they wrote which we didn't see
    """
    plt.figure(figsize=(8, 8))
    
    # Use ZScale for astronomical images
    interval = ZScaleInterval()
    vmin, vmax = interval.get_limits(data)
    norm = ImageNormalize(vmin=vmin, vmax=vmax, stretch=LinearStretch())
    
    plt.imshow(data, origin='lower', cmap='gray', norm=norm)
    plt.colorbar(label='Counts')
    plt.title(title)
    print(f"{title}: min={np.min(data):.1f}, max={np.max(data):.1f}, median={np.median(data):.1f}")
    print(f"Display range (ZScale): {vmin:.1f} to {vmax:.1f}")
    plt.show()

In [ ]:
bias = fits.getdata("data/calibration/bias/2025_10_04_bias_02.fits")

show_fits(bias, "Indivisual Bias Frame")

mean_bias = np.median(bias)

print(f"bias mean: {np.mean(mean_bias):.2f} ADU")

In [ ]:
bias_files = sorted(glob.glob(os.path.join(BIAS_DIR, "*.fits")))
bias_stack = np.stack([fits.getdata(f) for f in bias_files])
master_bias = np.median(bias_stack, axis=0)

trim = 200  # idk what this should be lol
master_bias = master_bias[trim:-trim, trim:-trim]

fits.writeto(os.path.join(MASTER_DIR, "master_bias.fits"), master_bias, overwrite=True)
show_fits(master_bias, "Master Bias Frame")
print(f"Master bias mean: {np.mean(master_bias):.2f} ADU")


In [ ]:
flat_files = sorted(glob.glob(os.path.join(FLAT_DIR, "*.fits")))

# Extracting filter names
filters = sorted(set([os.path.basename(f).split("_")[3] for f in flat_files]))
print("Detected filters:", filters)

bias = master_bias

for flt in filters:
    flt_files = [f for f in flat_files if f"_{flt}_" in f]
    flats = [(fits.getdata(f)[trim:-trim, trim:-trim] - bias) for f in flt_files]
    flats_norm = [f / np.mean(f) for f in flats]
    master_flat = np.median(np.stack(flats_norm), axis=0)
    master_flat /= np.mean(master_flat)
    
    out_path = os.path.join(MASTER_DIR, f"master_flat_{flt}.fits")
    fits.writeto(out_path, master_flat, overwrite=True)

    show_fits(master_flat, f"Master Flat ({flt}-filter)")


In [ ]:
master_bias = fits.getdata(os.path.join(MASTER_DIR, "master_bias.fits"))
# Choose appropriate filter manually for now
flt = "R"
master_flat = fits.getdata(os.path.join(MASTER_DIR, f"master_flat_{flt}.fits"))

transit_files = sorted(glob.glob(os.path.join(TRANSIT_DIR, "*.fits")))

for f in transit_files:
    data, hdr = fits.getdata(f, header=True)
    data = data[trim:-trim, trim:-trim]  # trim science frame
    reduced = (data - master_bias) / master_flat
    reduced /= hdr.get('EXPTIME', 1) # Exposure time normalisation

    out_path = os.path.join(OUTPUT_TRANSIT, os.path.basename(f))
    fits.writeto(out_path, reduced, hdr, overwrite=True)
    print(f"Reduced: {os.path.basename(f)}")

    # Optionally view before/after for one frame
    show_fits(data, "Raw science frame")
    show_fits(reduced, "Reduced science frame")
    break  # remove this once checked


In [ ]:
# Standard Star Analysis 
# Need to apply each filter manually

master_bias = fits.getdata(os.path.join(MASTER_DIR, "master_bias.fits"))
# Choose appropriate filter manually for now
flt = "R" # also B, V, I
master_flat = fits.getdata(os.path.join(MASTER_DIR, f"master_flat_{flt}.fits"))

standards_files = sorted(glob.glob(os.path.join(STANDARD_DIR, "*.fits")))

for s in standards_files:
    data, hdr = fits.getdata(s, header=True)
    data = data[trim:-trim, trim:-trim]  # trim science frame
    reduced = (data - master_bias) / master_flat
    reduced /= hdr.get('EXPTIME', 1) # Exposure time normalisation

    out_path = os.path.join(OUTPUT_STANDARDS, os.path.basename(s))
    fits.writeto(out_path, reduced, hdr, overwrite=True)
    print(f"Reduced: {os.path.basename(s)}")

    # Optionally view before/after for one frame
    show_fits(data, "Raw science frame")
    show_fits(reduced, "Reduced science frame")
    break  # remove this once checked